<a href="https://colab.research.google.com/github/newmno4967/AIFFEL_quest_cr/blob/master/Chart_Journeys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#차트 여정단
팀원: 백승호, 최창윤, 김민상  
  
1. 차트 여정단 프로젝트
    *   목표: 여행하고 싶은 지역의 에어비엔비 숙소들을 비교하여 점수를 매겨 우리가 직접 추천해주는 프로그램
    *   결과: 여행 목적지와 1박에 사용할수 있는 금액을 입력받아 조건에 맞는 에어비엔비 방중에서 점수가 높은순으로 10개를 출력함

2. 데이터 세트
    * 데이터는 110개의 도시(나라)와 각각의 도시의 listings.csv, calendar.csv, 	reviews.csv파일 이렇게 3개가 존재한다.
    * listings파일은 현재 도시의 전체적인 데이터(예: 숙소의 세부정보, 호스트의 세부정보, 리뷰의 점수, 가격 등)가 존재하여 가장 중요하고 많은 정보가 있다.
    * calendar파일은 숙소들이 예약이 가능한지를 보여주는 데이터가 있다.
    * reviews파일은 도시에 존재하는 모든 숙소들의 리뷰가 있는 데이터이다
    * 우리의 목적에 따라 listings파일중 필요한 데이터만 뽑아 쓰기로 결정했다.

3. 변수 분석
    * Listings 는 가격 포함 총 75개의 변수로 구성됨
    * 필요에 따라 데이터를 뽑아쓰기 위해 오리지널 데이터를 만들고 각각의 작업하는 목적에 따라 stay_rating, display_price, draw_map으로 3개로 나눠 각각의 필요한 데이터를 불러온다
    * stay_rating은 우리가 각각의 숙소에 점수를 매기기 위해 점수에 영향을주는 컬럼(id, host_response_time, host_response_rate, host_identity_verified, bathrooms, bedrooms, amenities, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_communication, review_scores_location, review_scores_value)을 사용한다.
    * display_price는 stay_rating에서 매긴 점수에 따라 상위 10개의 숙소를 보여주기 위해 필요한 컬럼(id, name, price, listing_url)을 사용한다
    * draw_map은 우리가 뽑은 숙소를 지도에서 보여주기위해 만들었기 때문에
    필요한 컬럼(id, chart_rating, listing_url, latitude, longitude, name)을 사용한다.

4. 코드
    1. 우리가 여행을 갈 목적지와 1박에 사용가능한 최대 금액을 입력받는다.
    2. 여행을 갈 지역의 데이터를 불러온다.
    3. 과정들에 필요한 데이터들을 따로 저장하고 만들어둔다.
    4. stay_rating과정을 시작하고 결측치들은 0으로 추가한다.
    5. 데이터가 많기 때문에 이상치가있으면 전부 제거
    6. stay_rating에 chart_rating칼럼 추가
    7. chart_rating칼럼에 각각의 기준에 따라 점수를 조금씩 추가한다.
    8. 만들어둔 display_price에 chart_rating컬럼을 붙여넣는다.
    9. price컬럼을 숫자형으로 바꿔준다.
    10. 입력된 1박에 사용할수 있는 금액보다 비싼 숙소들은 제외시킨다.
    11. 사용자에게 조건에 만족하는 숙소중 점수가 높은 10개의 숙소를 주요정보와 보여준다.
    12. 10개의 숙소를 지도에 아이콘으로 하여 보여준다.





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 사용할 라이브러리 불러오기
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np

In [3]:
country = input('검색하고 싶은 도시를 입력하세요 : ')
price = int(input('1박에 사용할수있는 금액을 입력하세요(달러) : '))

검색하고 싶은 도시를 입력하세요 : Belize
1박에 사용할수있는 금액을 입력하세요(달러) : 1000


In [4]:
airbnb = f"/content/drive/MyDrive/airbnb/{country}_listings.csv"
original_data = pd.read_csv(airbnb)

In [5]:
#rating변수에 rating에서 사용할 데이터만 저장
rating = ['id', 'host_response_time', 'host_response_rate', 'host_identity_verified', 'bathrooms', 'bedrooms', 'amenities', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_location', 'review_scores_value']
display = ['id', 'name', 'price', 'listing_url']
stay_rating = original_data[rating]
display_price = original_data[display]

In [6]:
#stay_rating에서 null값을 0으로 치환
stay_rating = stay_rating.fillna(0)

In [7]:
#host_response_time컬럼에서 within a few hours, within an hour, a few days or more, within a day를 제외한 이상치 데이터 제거
outliers =  (stay_rating['host_response_time']== 0) | (stay_rating['host_response_time'] == 'within a few hours') | (stay_rating['host_response_time'] == 'within an hour') | (stay_rating['host_response_time'] == 'a few days or more') | (stay_rating['host_response_time'] == 'within a day')

stay_rating = stay_rating[outliers]

In [8]:
#host_response_rate컬럼에서 %를 제거하고 float형식으로 변환 및 Null값을 0으로 치환
stay_rating['host_response_rate'] = stay_rating['host_response_rate'].str.rstrip('%').astype(float)
stay_rating['host_response_rate'] = stay_rating['host_response_rate'].fillna(0)

In [9]:
#host_identity_verified컬럼에서 t와 f가 아닌 이상치 제거(0은 Null값 채우는 용)
outliers =  (stay_rating['host_identity_verified']=='t') | (stay_rating['host_identity_verified'] == 'f') | (stay_rating['host_identity_verified']== 0)

stay_rating = stay_rating[outliers]

In [10]:
#ast라이브러리에서 literal_eval불러오기
from ast import literal_eval

#amenities컬럼을 list형태로 변환
stay_rating['amenities'] = stay_rating['amenities'].apply(literal_eval)

In [11]:
#review_scores_rating칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_rating'] <= 5)

stay_rating = stay_rating[outliers]

In [12]:
#review_scores_accuracy칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_accuracy'] <= 5)

stay_rating = stay_rating[outliers]

In [13]:
#review_scores_cleanliness칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_cleanliness'] <= 5)

stay_rating = stay_rating[outliers]

In [14]:
#review_scores_communication칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_communication'] <= 5)

stay_rating = stay_rating[outliers]

In [15]:
#review_scores_location칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_location'] <= 5)

stay_rating = stay_rating[outliers]

In [16]:
#review_scores_value칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_value'] <= 5)

stay_rating = stay_rating[outliers]

In [17]:
#stay_rating에 chart_rating칼럼 추가
stay_rating['chart_rating'] = 0

In [18]:
#host_response_time에 within an hour면 0.3점, within a few hours면 0.2점, within a day면 0.1점 나머지 값은 0점을 chart_rating에 추가한다.
conditions = [
    stay_rating['host_response_time'] == "within a day",
    stay_rating['host_response_time'] == "within a few hours",
    stay_rating['host_response_time'] == "within an hour"
]
values = [0.1, 0.2, 0.3]

stay_rating['chart_rating'] += np.select(conditions, values, default=0)

In [19]:
#host_response_rate가 100%면 0.2점 40%미만이면 0점 40%~99%면 0.1점을chart_rating에 추가
conditions = [
    stay_rating['host_response_rate'] == 100,
    stay_rating['host_response_rate'] < 40
]
values = [0.2, 0]

stay_rating['chart_rating'] += np.select(conditions, values, default=0.1)

In [20]:
#host_identity_verified(호스트 인증)이 되어있으면 0.5점을 chart_rating에 점수 추가
conditions = [
    stay_rating['host_identity_verified'] == 't',
    stay_rating['host_identity_verified'] == 'f'
]
values = [1, 0]

stay_rating['chart_rating'] += np.select(conditions, values, default=0)

In [21]:
#bathrooms의 개수가 0개면 0점, 1개면 1점 1개 이상이면 1.2점을 chart_rating에 점수 추가
conditions = [
    stay_rating['bathrooms'] == 0,
    stay_rating['bathrooms'] == 1
]
values = [0, 1]

stay_rating['chart_rating'] += np.select(conditions, values, default=1.2)

In [22]:
#bedrooms의 개수가 0개면 0점, 1개면 1점 1개 이상이면 1.2점을 chart_rating에 점수 추가
conditions = [
    stay_rating['bedrooms'] == 0,
    stay_rating['bedrooms'] == 1
]
values = [0, 1]

stay_rating['chart_rating'] += np.select(conditions, values, default=1.2)

In [23]:
#amenities에 있는 편의시설 1개당 0.05점씩 최대 1점을 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['amenities'].str.len().apply(lambda x: 1 if x >= 20 else x / 20)

In [24]:
#review_scores_rating점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_rating'].apply(lambda x : x/5)

In [25]:
#review_scores_accuracy점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_accuracy'].apply(lambda x : x/5/5)

In [26]:
#review_scores_cleanliness점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_cleanliness'].apply(lambda x : x/5)

In [27]:
#review_scores_communication점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_communication'].apply(lambda x : x/5/2)

In [28]:
#review_scores_location점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_location'].apply(lambda x : x/5*1.2)

In [29]:
#review_scores_value점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_value'].apply(lambda x : x/5*1.2)

In [30]:
#display_price에 stay_rating에서 만든 chart_rating복사하여 붙이기
display_price['chart_rating'] = stay_rating['chart_rating']

<ipython-input-30-4738a7a6156c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  display_price['chart_rating'] = stay_rating['chart_rating']


In [31]:
#chart_rating컬럼을 str형식에서 특수문자를 없에고 float형식으로 바꾼다
display_price['price'] = display_price['price'].str.replace('[^\d.]', '', regex=True).astype(float)
display_price['price'] = display_price['price'].fillna(0)

<ipython-input-31-19453f34860a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  display_price['price'] = display_price['price'].str.replace('[^\d.]', '', regex=True).astype(float)
<ipython-input-31-19453f34860a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  display_price['price'] = display_price['price'].fillna(0)


In [32]:
#display_price에서 사용자가 입력한 금액보다 큰 금액이 필요한 것들 지우기
outliers_price = (display_price['price'] < price)
display_price = display_price[outliers_price]

In [33]:
#사용자에게 중요한 정보 보여주기
draw_map = display_price[['id', 'chart_rating', 'listing_url']].sort_values('chart_rating', ascending=False).head(10)
display_price.sort_values('chart_rating', ascending=False).head(10)

,id,name,price,listing_url,chart_rating
2820,1036888742536435010,Hopkins Beach Retreat - Balcony House,383.0,https://www.airbnb.com/rooms/1036888742536435010,10.0
3128,1184912107206660549,Island Magic Ground Floor Villas,591.0,https://www.airbnb.com/rooms/1184912107206660549,10.0
2817,1036319666195220709,Hopkins Beach Retreat - Pool House,322.0,https://www.airbnb.com/rooms/1036319666195220709,10.0
2138,709436191435164802,Villa 99-Octopus/Safari-Beachfront-Pool-3 Bedroom,652.0,https://www.airbnb.com/rooms/709436191435164802,10.0
2136,709436031522601997,Villa 99-Safari-Beachfront Villa-Pool-2 Bedroom,453.0,https://www.airbnb.com/rooms/709436031522601997,10.0
1059,34834833,Caye Reef Ocean view 2nd floor 2 bedroom apart...,525.0,https://www.airbnb.com/rooms/34834833,10.0
2123,702331491534048337,"#4 Beach condo 2BR/2BA: balcony, pool, kayaks!",405.0,https://www.airbnb.com/rooms/702331491534048337,10.0
2771,1018924018764494214,Luxury 2 Bedroom 2 Bathroom Beachside Villa,943.0,https://www.airbnb.com/rooms/1018924018764494214,10.0
2759,1012393498468742256,Luxurious Newly renovated 2bed condo on the Sea,486.0,https://www.airbnb.com/rooms/1012393498468742256,10.0
2751,1003576361587377965,Unique Oceanfront Private Estate by ALOM,827.0,https://www.airbnb.com/rooms/1003576361587377965,10.0


In [34]:
from IPython.display import display, HTML
from folium.plugins import MarkerCluster
import folium

HTML('<style>.output {flex-direction: row;}</style>')

geo_display= ['latitude','longitude', 'name']

test = draw_map
test[['latitude','longitude', 'name']] = original_data[['latitude','longitude', 'name']]

latitude = test['latitude'].mean()
longitude= test['longitude'].mean()

rating_map= folium.Map(location=[latitude, longitude], zoom_start=9)

rating_Marker = MarkerCluster().add_to(rating_map)

def getcolor(rating) :
  if rating >= 9:
    return 'red'
  elif rating >= 7:
    return 'blue'
  elif rating >= 5:
    return 'green'
  else :
    return 'yellow'

for _, row in test.iterrows():

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']} - url: {row['listing_url']}",
        tooltip=row['name'],
        icon=folium.Icon(color=getcolor(row['chart_rating']))
    ).add_to(rating_map)

display(display_price.sort_values('chart_rating', ascending=False).head(10))
rating_map

,id,name,price,listing_url,chart_rating
2820,1036888742536435010,Hopkins Beach Retreat - Balcony House,383.0,https://www.airbnb.com/rooms/1036888742536435010,10.0
3128,1184912107206660549,Island Magic Ground Floor Villas,591.0,https://www.airbnb.com/rooms/1184912107206660549,10.0
2817,1036319666195220709,Hopkins Beach Retreat - Pool House,322.0,https://www.airbnb.com/rooms/1036319666195220709,10.0
2138,709436191435164802,Villa 99-Octopus/Safari-Beachfront-Pool-3 Bedroom,652.0,https://www.airbnb.com/rooms/709436191435164802,10.0
2136,709436031522601997,Villa 99-Safari-Beachfront Villa-Pool-2 Bedroom,453.0,https://www.airbnb.com/rooms/709436031522601997,10.0
1059,34834833,Caye Reef Ocean view 2nd floor 2 bedroom apart...,525.0,https://www.airbnb.com/rooms/34834833,10.0
2123,702331491534048337,"#4 Beach condo 2BR/2BA: balcony, pool, kayaks!",405.0,https://www.airbnb.com/rooms/702331491534048337,10.0
2771,1018924018764494214,Luxury 2 Bedroom 2 Bathroom Beachside Villa,943.0,https://www.airbnb.com/rooms/1018924018764494214,10.0
2759,1012393498468742256,Luxurious Newly renovated 2bed condo on the Sea,486.0,https://www.airbnb.com/rooms/1012393498468742256,10.0
2751,1003576361587377965,Unique Oceanfront Private Estate by ALOM,827.0,https://www.airbnb.com/rooms/1003576361587377965,10.0
